In [1]:
import pytest
import asyncio
from httpx import AsyncClient

@pytest.mark.asyncio
async def test_concurrent_requests():
    async with AsyncClient(app=app, base_url="http://localhost:8008") as client:
        # Список для хранения всех отправленных запросов
        requests = []

        # Отправляем 100 запросов асинхронно
        for _ in range(100):
            # Создаем корутину для отправки запроса и добавляем ее в список
            request_coroutine = client.post("/llm", json={"user_id": 123, "question_string": "Test question"})
            requests.append(request_coroutine)

        # Ожидаем завершения всех запросов
        responses = await asyncio.gather(*requests)

        # Проверяем статус-коды всех ответов
        for response in responses:
            assert response.status_code == 200


ModuleNotFoundError: No module named 'pytest'

# генерация ключей

In [25]:
import secrets

# Генерация случайного ключа
def generate_api_key():
    # Генерация случайной строки из символов шестнадцатеричного формата
    api_key = secrets.token_hex(32)  # Вы можете выбрать любую длину ключа, 32 байта (64 символа шестнадцатеричного формата) в данном случае
    return api_key

# Генерация API ключа
api_key = generate_api_key()
api_key

'716beeee49e8981b54619f960579f798f40dba42e1cf88260a77ddee6ae7b94f'

# POST

In [4]:
import requests
import json

URL = "http://localhost:8008/llm"
POST_JSON = {
    "user_id": 1050,
    "question_string": "Чем ты можешь быть мне полезен?"
}
r = requests.post(
    url=URL,
    headers={
        "Content-Type": "application/json",
        "API": "131e12aa46252f4da6920dd2feccc94978688eab3a96337ba4b67a945eac1308"
    },
    json=POST_JSON
)

response = r.json()
response

{'user_id': 1050,
 'date': '2024-02-07',
 'time': '10:13:02',
 'client_ip': '127.0.0.1',
 'ai_response': 'Я могу помочь вам в различных областях, таких как математика, физика, химия, история, литература, география, английский язык, французский язык, испанский язык, немецкий язык, китайский язык, японский язык, программирование, компьютерные науки, технологии, экономика, финансы, маркетинг, психология, медицина, юриспруденция, право, политика, международные отношения, управление, предпринимательство, проектирование, инженерия, архитектура, дизайн, искусство, музыка, спорт, фитнес, здоровье, питание, кулинария, рецепты, домашнее хозяйство, ремонт, строительство, автомобили, мотоциклы, велосипеды, мотороллеты, катание на роликах, катание на колясках, катание на снегоходе, катание на лыжах, катание на сноуборде, катание на горнолыжных дорогах, катание на катере, катание на парусниках, катание на яхтах, катание на каноэ, катание на байдарках, катание на каяках, катание на лодках, катание на

# отладка чат-бота

In [2]:
from lib.vector_db_setup import get_texts, upload_to_vectorstore, vectorstore_query, get_chroma_client
import chromadb
from chromadb.config import Settings

texts = get_texts()

upload_to_vectorstore(texts, collection_name='book')

# Подключаемся к векторному хранилищу
chroma_client = get_chroma_client()
# Получаем коллекцию
collection = chroma_client.get_collection('book')

Применяю датасет по умолчаюнию: ./SOURCE_DOCUMENTS/questions_stolot.csv
Датасет успешно загружен
Загрузка модели для эмбеддингов: SUCCESS
Подключение к векторной БД: SUCCESS
Создание коллекции book: SUCCESS
Загружаю данные в векторную БД...
Загрузка данных в векторную БД: SUCCESS
Подключение к векторной БД: SUCCESS


In [4]:
question = 'Почему я не могу купить билет?'
vector_db_response = vectorstore_query(
    collection,
    question = question,
    n_results=3
)

print(f'============ QUESTION: ============\n{question}')
print(f'============ VECTOR DB RESPONSE: ============\n{vector_db_response}')

============ QUESTION: ============
Почему я не могу купить билет?
============ VECTOR DB RESPONSE: ============
Возможно, Вы исчерпали ежемесячные лимиты по Кошельку. Вам необходимо пройти идентификацию на сайте Столото или ожидать первого числа нового месяца, когда лимиты обновятся.


# llm

In [5]:
# Подключаемся к векторному хранилищу
chroma_client = get_chroma_client()
# Получаем коллекцию
collection = chroma_client.get_collection('book')

Подключение к векторной БД: SUCCESS


In [8]:
from peft import PeftConfig
from transformers import AutoTokenizer
from lib.llm_setup import initialize_model, generate_llm_response

lora_adapter = "IlyaGusev/saiga_mistral_7b"
config = PeftConfig.from_pretrained(lora_adapter)
base_model = config.base_model_name_or_path

# Настройка токенизатора
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right' # ???

model = initialize_model(
    base_model=base_model,
    lora_adapter=lora_adapter
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
question = 'Почему я не могу купить билет?'

output_message = generate_llm_response(
    question=question,
    model=model,
    collection=collection,
    tokenizer=tokenizer
)